# Doc2Vecのラベル付与に関する調査

こちらを参考にしました。


- Doc2Vec tutorial using Gensim

　　https://medium.com/@klintcho/doc2vec-tutorial-using-gensim-ab3ac03d3a1
  
- Doc2vec tutorial

　　https://rare-technologies.com/doc2vec-tutorial/


サンプルごとにユニークなIDを使ってラベルを付与してみました。

この結果、すべてのサンプルについてのベクトルが、モデル内に保持されるようにはなりました。

ただし、<a href="11-doc2vec-tutor.ipynb"><b>前回検証時</b></a>とは異なり、まったく適切な回答が得られなくなってしまいました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

既存実装どおり、Nlangクラスを使い、コーパス（単語が半角スペースで区切られた文字列）を生成します。

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}
learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])
ids = np.array(learning_training_messages['id'])

2017/05/16 PM 02:10:22 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/16 PM 02:10:22 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang.batch_split(_sentences)

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

ただし、今回は学習セットの各サンプルに、ユニークなタグを付与します。

（便宜上、[answer\_id]\_[id] の形式としています）

In [5]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [6]:
def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences_by_ids(separated_sentences, answer_ids, ids):
    for idx, (doc, answer_id, ID) in enumerate(zip(separated_sentences, answer_ids, ids)):
        name = '%d_%d' % (answer_id, ID)
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [7]:
'''
    学習セットの全レコードがユニークになるよう、
    ラベルを、学習セットのIDごとに付与する
'''
sentences = corpus_to_sentences_by_ids(_separated_sentences, answer_ids, ids)

In [8]:
sentence_list = list(sentences)

In [9]:
model = Doc2Vec(size=500, min_count=1, iter=200)

In [10]:
model.build_vocab(sentence_list)

In [11]:
model.train(sentence_list)

13680963

In [12]:
model_path = 'prototype/better_algorithm/doc2vec_unique.model'

model.save(model_path)

In [13]:
'''
    モデル内に保持されているベクトルの数を取得
    （ラベルをレコードIDにすると、サンプルの数が戻る。
    　同一回答IDのサンプルのベクトルが
    　上書きされなくなることを確認）
'''
len(model.docvecs)

16288

### (2-4) 予測処理

ラベルをサンプルのIDとした（＝サンプルごとにユニークなラベルを付与した）場合、正しい回答が得られなくなってしまうようです。

In [14]:
loaded_model = models.Doc2Vec.load(model_path)

In [15]:
len(loaded_model.docvecs)

16288

In [16]:
'''
    予測処理にかけるコーパスを生成
'''
test_corpus1 = ['情報','システム','アドレス']
test_corpus2 = ['マウス','破損']

In [17]:
'''
    情報システムのアドレス（正解＝7040、ID＝2338234）

    （4526＝「パワーポイント 動かない」）
'''
loaded_model.delete_temporary_training_data()

inferred_vector1 = loaded_model.infer_vector(test_corpus1)
loaded_model.docvecs.most_similar([inferred_vector1])

[('4526_2339018', 0.7372329235076904),
 ('4526_2339014', 0.7355606555938721),
 ('4526_2323406', 0.7326104640960693),
 ('4526_2323402', 0.7324880361557007),
 ('4526_2339019', 0.7242528796195984),
 ('4526_2327122', 0.7148222923278809),
 ('4526_2327124', 0.7143131494522095),
 ('4526_2334048', 0.7126641273498535),
 ('4590_2329470', 0.7113333344459534),
 ('4450_2332279', 0.7079724073410034)]

In [18]:
'''
    マウス破損（正解＝4458）

    （4581＝「パワーポイント スライド 破損」）
'''
inferred_vector2 = loaded_model.infer_vector(test_corpus2)
loaded_model.docvecs.most_similar(positive=[inferred_vector2])

[('4581_2329057', 0.766130805015564),
 ('4581_2335978', 0.7354090213775635),
 ('4581_2339195', 0.7279403209686279),
 ('4581_2323583', 0.7247350215911865),
 ('4480_2325660', 0.7189759612083435),
 ('4581_2335991', 0.7086151838302612),
 ('4480_2325647', 0.7047380208969116),
 ('4480_2325662', 0.704692006111145),
 ('4522_2326954', 0.7039743661880493),
 ('4480_2324459', 0.7011843323707581)]